In [43]:
import numpy as np
import pandas as pd
import xgboost as xgb
from datetime import datetime,timedelta
from numpy.matlib import repmat
def mape_nparray(y,d):
    c=d
    rec=np.sum(np.abs(y-c)/c)/len(c)
    return rec

def mape(y,d):
    c=d.get_label()
    result=np.sum(np.abs(y-c)/c)/len(c)
    return "mape",result

In [44]:
tmptime=datetime(2016,7,19)
timeseries=[]
while(tmptime<datetime(2016,11,1)):
    timeseries=timeseries+[str(tmptime)]
    tmptime=tmptime+timedelta(minutes=20)
    
idframe=pd.DataFrame({"intersection_id":["A","A","B","B","C","C"],
              "tollgate_id":[2,3,1,3,1,3],
             })
idframe["hhh"]=1
timeframe=pd.DataFrame(timeseries)
timeframe["hhh"]=1
twframe=pd.merge(idframe,timeframe,how="outer")
twframe=twframe.iloc[:,[0,1,3]]
twframe.rename(columns={0:'datetime'}, inplace = True)
twframe["datetime"]=pd.to_datetime(twframe["datetime"])

In [45]:
def data_avg(train):
    q=train.groupby(["month","date","hour","minute","intersection_id","tollgate_id"])["travel_time"].mean()
    p=train.drop(["travel_time","datetime","starting_time"],axis=1).drop_duplicates().sort(columns=["month","date","hour","minute","intersection_id","tollgate_id"])
    p["travel_time"]=q.values
    cols=list(p)
    cols.insert(2, cols.pop(cols.index('travel_time')))
    p= p.ix[:, cols]
    return p

In [46]:
def select_val_knn(data):
    select_time2=((data["hour"]<10)&(data["hour"]>7))|((data["hour"]<19)&(data["hour"]>16))
    select_time1=((data["hour"]<8)&(data["hour"]>5))|((data["hour"]<17)&(data["hour"]>14))
    
    t1=(data["month"]==10)&(data["date"]==25)&select_time2
    t2=(data["month"]==10)&(data["date"]==26)&select_time2
    t3=(data["month"]==10)&(data["date"]==27)&select_time2
    t4=(data["month"]==10)&(data["date"]==28)&select_time2
    t5=(data["month"]==10)&(data["date"]==29)&select_time2
    t6=(data["month"]==10)&(data["date"]==30)&select_time2
    t7=(data["month"]==10)&(data["date"]==31)&select_time2
    
    e1=(data["month"]==10)&(data["date"]==25)&select_time1
    e2=(data["month"]==10)&(data["date"]==26)&select_time1
    e3=(data["month"]==10)&(data["date"]==27)&select_time1
    e4=(data["month"]==10)&(data["date"]==28)&select_time1
    e5=(data["month"]==10)&(data["date"]==29)&select_time1
    e6=(data["month"]==10)&(data["date"]==30)&select_time1
    e7=(data["month"]==10)&(data["date"]==31)&select_time1
    
    dt1=data[t1].copy()
    dt2=data[t2].copy()
    dt3=data[t3].copy()
    dt4=data[t4].copy()
    dt5=data[t5].copy()
    dt6=data[t6].copy()
    dt7=data[t7].copy()
    
    
    de1=data[e1].copy()
    de2=data[e2].copy()
    de3=data[e3].copy()
    de4=data[e4].copy()
    de5=data[e5].copy()
    de6=data[e6].copy()
    de7=data[e7].copy()


    
    dtrain=data[(data["month"]*31+data["date"])<335].copy()
    dpred=pd.concat([dt1,dt2,dt3,dt4,dt5,dt6,dt7])
    dtest=pd.concat([de1,de2,de3,de4,de5,de6,de7])
    return dtrain,dtest,dpred,data

In [47]:
def select_val(data):
    select_time2=((data["hour"]<10)&(data["hour"]>7))|((data["hour"]<19)&(data["hour"]>16))
    select_time1=((data["hour"]<8)&(data["hour"]>5))|((data["hour"]<17)&(data["hour"]>14))
    
    t1=(data["month"]==10)&(data["date"]==11)&select_time2
    t2=(data["month"]==10)&(data["date"]==12)&select_time2
    t3=(data["month"]==10)&(data["date"]==13)&select_time2
    t4=(data["month"]==10)&(data["date"]==14)&select_time2
    t5=(data["month"]==10)&(data["date"]==15)&select_time2
    t6=(data["month"]==10)&(data["date"]==16)&select_time2
    t7=(data["month"]==10)&(data["date"]==17)&select_time2
    
    e1=(data["month"]==10)&(data["date"]==11)&select_time1
    e2=(data["month"]==10)&(data["date"]==12)&select_time1
    e3=(data["month"]==10)&(data["date"]==13)&select_time1
    e4=(data["month"]==10)&(data["date"]==14)&select_time1
    e5=(data["month"]==10)&(data["date"]==15)&select_time1
    e6=(data["month"]==10)&(data["date"]==16)&select_time1
    e7=(data["month"]==10)&(data["date"]==17)&select_time1
    
    dt1=data[t1].copy()
    dt2=data[t2].copy()
    dt3=data[t3].copy()
    dt4=data[t4].copy()
    dt5=data[t5].copy()
    dt6=data[t6].copy()
    dt7=data[t7].copy()
    
    
    de1=data[e1].copy()
    de2=data[e2].copy()
    de3=data[e3].copy()
    de4=data[e4].copy()
    de5=data[e5].copy()
    de6=data[e6].copy()
    de7=data[e7].copy()


    
    dtrain=data[(data["month"]*31+data["date"])<321].copy()
    dpred=pd.concat([dt1,dt2,dt3,dt4,dt5,dt6,dt7])
    dtest=pd.concat([de1,de2,de3,de4,de5,de6,de7])
    return dtrain,dtest,dpred,data

In [48]:
def data_to_TrainAndVal(data,datao):
      
    data_starting_time=data["starting_time"].str.split(' ',expand=True)
    data_starting_time_date=data_starting_time[0].str.split('-',expand=True)
    data_starting_time_time=data_starting_time[1].str.split(':',expand=True)
    data["month"]=data_starting_time_date[1].apply(int)
    data["date"]=data_starting_time_date[2].apply(int)
    data["hour"]=data_starting_time_time[0].apply(int)
    data["minute"]=data_starting_time_time[1].apply(int)
    data["minute"]=(data["minute"]/20).apply(int)*20
    data["number_intersection_id"]=data["intersection_id"].replace(["A","B","C"],[1,2,3])
    data_std=datao.groupby(["intersection_id","tollgate_id","hour"])["travel_time"].std().reset_index().rename(columns={"travel_time":"travel_time_std"})
    data_mean=datao.groupby(["intersection_id","tollgate_id","hour"])["travel_time"].mean().reset_index().rename(columns={"travel_time":"travel_time_mean"})
    data=pd.merge(data,data_std,on=["intersection_id","tollgate_id","hour"])
    data=pd.merge(data,data_mean,on=["intersection_id","tollgate_id","hour"])
    
    data["holiday_before1"]=1*(((data["month"]==9)&(data["date"]==14))\
                            |((data["month"]==9)&(data["date"]==30)))
    data["holiday_before2"]=1*(((data["month"]==9)&(data["date"]==15))\
                            |((data["month"]==10)&(data["date"]==1)))
    data["holiday_after1"]=1*(((data["month"]==9)&(data["date"]==17))\
                            |((data["month"]==10)&(data["date"]==7)))
    data["holiday_after2"]=1*(((data["month"]==9)&(data["date"]==18))\
                            |((data["month"]==10)&(data["date"]==8)))
    data["holiday1"]=1*((data["month"]==9)&(data["date"]>15)&(data["date"]<19))
    data["holiday2"]=1*((data["month"]==10)&(data["date"]>0)&(data["date"]<8))
    data["workholiday"]=1*(((data["month"]==9)&(data["date"]==19))\
                        |((data["month"]==10)&(data["date"]==9))\
                        |((data["month"]==10)&(data["date"]==10)))
    data["holiday"]=data["holiday1"]+data["holiday2"]
   
    
    data["datetime"]=pd.to_datetime(data["starting_time"])
    data["weekday"]=data["datetime"].dt.weekday
    
    #one_hot A,B,C
    data["ifA"]=data["intersection_id"].replace(["A","B","C"],[1,0,0])
    data["ifB"]=data["intersection_id"].replace(["A","B","C"],[0,1,0])
    data["ifC"]=data["intersection_id"].replace(["A","B","C"],[0,0,1])

    #one_hot 1,2,3
    data["if1"]=data["tollgate_id"].replace([1,2,3],[1,0,0])
    data["if2"]=data["tollgate_id"].replace([1,2,3],[0,1,0])
    data["if3"]=data["tollgate_id"].replace([1,2,3],[0,0,1])
    
    #one_hot weekday
    data["wk1"]=data["weekday"].replace(range(0,7),[1,0,0,0,0,0,0])
    data["wk2"]=data["weekday"].replace(range(0,7),[0,1,0,0,0,0,0])
    data["wk3"]=data["weekday"].replace(range(0,7),[0,0,1,0,0,0,0])
    data["wk4"]=data["weekday"].replace(range(0,7),[0,0,0,1,0,0,0])
    data["wk5"]=data["weekday"].replace(range(0,7),[0,0,0,0,1,0,0])
    data["wk6"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,1,0])
    data["wk7"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,0,1])
    
    data["time_number"]=data["hour"]*60+data["minute"]
    data["date_number"]=(data["datetime"]-datetime(2016,7,19)).dt.days
    

    dtrain,dtest,dpred,ddata=select_val(data)
    data_validation=dpred
    q=data_validation.groupby(["month","date","hour","minute","intersection_id","tollgate_id"])["travel_time"].mean()
    p=data_validation.drop(["travel_time","datetime","starting_time","time_number"],axis=1).drop_duplicates().sort(columns=["month","date","hour","minute","intersection_id","tollgate_id"])
    p["travel_time"]=q.values
    cols=list(p)
    cols.insert(2, cols.pop(cols.index('travel_time')))
    p= p.ix[:, cols]
    data_validation=p

    

    return dtrain,dtest,data_validation,ddata

In [50]:
data1 = pd.read_csv('../RawData/trajectories(table 5)_training.csv')
data2 = pd.read_csv('../RawData/trajectories(table 5)_test1.csv')
data=pd.concat([data1,data2])
data=data.iloc[:,[0,1,3,5]]
dtrain,dtest,val,ddata=data_to_TrainAndVal(data,data)
datao=ddata.copy()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:58: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [51]:
train=pd.concat([dtrain,dtest]).ix[:,list(val)]

In [52]:
feature=[2]+range(17,19)+[3]+range(19,25)+[4,5]
x=train.copy().iloc[:,feature]
xx=x.copy().drop(['travel_time'],axis=1)
xy=(x['travel_time'])

z=val.copy().iloc[:,feature]
zx=z.drop(['travel_time'],axis=1)
zy=(z['travel_time'])
z

,travel_time,holiday,weekday,month,ifA,ifB,ifC,if1,if2,if3,date,hour
19526,68.092500,0,1,10,1,0,0,0,1,0,11,8
16143,146.161250,0,1,10,1,0,0,0,0,1,11,8
22255,86.560000,0,1,10,0,1,0,1,0,0,11,8
21513,142.522500,0,1,10,0,1,0,0,0,1,11,8
20440,158.305000,0,1,10,0,0,1,1,0,0,11,8
19538,75.023636,0,1,10,1,0,0,0,1,0,11,8
16151,168.431818,0,1,10,1,0,0,0,0,1,11,8
22257,127.100000,0,1,10,0,1,0,1,0,0,11,8
21517,93.000000,0,1,10,0,1,0,0,0,1,11,8
20442,148.323333,0,1,10,0,0,1,1,0,0,11,8


In [53]:
xlf = xgb.XGBRegressor(max_depth=6, 
                        learning_rate=0.1,
                        n_estimators=4000, 
                        silent=True, 
                        objective="reg:linear", 
    
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=2, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=1, 
                        missing=None)
xlf.fit(xx,xy, eval_metric=mape,verbose = True, eval_set = [(zx, zy)],early_stopping_rounds=20)
limit=xlf.best_iteration+1

[0]	validation_0-rmse:143.648	validation_0-mape:0.890495
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 20 rounds.
[1]	validation_0-rmse:132.423	validation_0-mape:0.800763
[2]	validation_0-rmse:122.077	validation_0-mape:0.716464
[3]	validation_0-rmse:112.742	validation_0-mape:0.637947
[4]	validation_0-rmse:104.272	validation_0-mape:0.56715
[5]	validation_0-rmse:97.2132	validation_0-mape:0.505776
[6]	validation_0-rmse:91.7988	validation_0-mape:0.458376
[7]	validation_0-rmse:86.2372	validation_0-mape:0.409313
[8]	validation_0-rmse:81.8563	validation_0-mape:0.369313
[9]	validation_0-rmse:77.9331	validation_0-mape:0.332062
[10]	validation_0-rmse:74.6573	validation_0-mape:0.301581
[11]	validation_0-rmse:71.3624	validation_0-mape:0.273257
[12]	validation_0-rmse:69.1875	validation_0-mape:0.25502
[13]	validation_0-rmse:67.2292	validation_0-mape:0.239488
[14]	validation_0-rmse:65.4548	validation

In [54]:
def adjust(target,a):
    l=target.copy()
    l["travel_time"]=l["travel_time"]\
                    +a[0]*((l["ifC"]==1)&(l["if3"]==1)&((l["hour"]==8)|(l["hour"]==9)))*l["travel_time"]
    l["travel_time"]=l["travel_time"]\
                    +a[1]*((l["ifC"]==1)&(l["if3"]==1)&((l["hour"]==17)|(l["hour"]==18)))*l["travel_time"]

    l["travel_time"]=l["travel_time"]\
                    +a[2]*((l["ifC"]==1)&(l["if1"]==1)&((l["hour"]==8)|(l["hour"]==9)))*l["travel_time"]
    l["travel_time"]=l["travel_time"]\
                    +a[3]*((l["ifC"]==1)&(l["if1"]==1)&((l["hour"]==17)|(l["hour"]==18)))*l["travel_time"]

    l["travel_time"]=l["travel_time"]\
                    +a[4]*((l["ifA"]==1)&(l["if2"]==1)&((l["hour"]==8)|(l["hour"]==9)))*l["travel_time"]
    l["travel_time"]=l["travel_time"]\
                    +a[5]*((l["ifA"]==1)&(l["if2"]==1)&((l["hour"]==17)|(l["hour"]==18)))*l["travel_time"]

    l["travel_time"]=l["travel_time"]\
                    +a[6]*((l["ifA"]==1)&(l["if3"]==1)&((l["hour"]==8)|(l["hour"]==9)))*l["travel_time"]
    l["travel_time"]=l["travel_time"]\
                    +a[7]*((l["ifA"]==1)&(l["if3"]==1)&((l["hour"]==17)|(l["hour"]==18)))*l["travel_time"]

    l["travel_time"]=l["travel_time"]\
                    +a[8]*((l["ifB"]==1)&(l["if1"]==1)&((l["hour"]==8)|(l["hour"]==9)))*l["travel_time"]
    l["travel_time"]=l["travel_time"]\
                    +a[9]*((l["ifB"]==1)&(l["if1"]==1)&((l["hour"]==17)|(l["hour"]==18)))*l["travel_time"]

    l["travel_time"]=l["travel_time"]\
                    +a[10]*((l["ifB"]==1)&(l["if3"]==1)&((l["hour"]==8)|(l["hour"]==9)))*l["travel_time"]
    l["travel_time"]=l["travel_time"]\
                    +a[11]*((l["ifB"]==1)&(l["if3"]==1)&((l["hour"]==17)|(l["hour"]==18)))*l["travel_time"]
    return l

In [55]:
pred=xlf.predict(zx,ntree_limit=limit)
l=val.copy()
adjust_para=np.array(range(0,12))*0.0
best_para=np.array(range(0,12))*0.0

bestscore=999


for i in range(0,12):
    tmp=list((np.array(range(0,61))-30)/200.0)
    for j in range(0,len(tmp)):
        tmp_para=best_para.copy()
        tmp_para[i]=tmp[j]
        l["travel_time"]=pred
        l=adjust(l,tmp_para)
        score=mape_nparray(l["travel_time"],zy)
        if (score<bestscore):
            bestscore=score
            best_para=tmp_para.copy()

    print(" i= ",i," best= ",bestscore)
    
pred=xlf.predict(zx,ntree_limit=limit)
l=val.copy()
l["travel_time"]=pred
l=adjust(l,best_para)

(' i= ', 0, ' best= ', 0.19038910956516578)
(' i= ', 1, ' best= ', 0.18964660350997192)
(' i= ', 2, ' best= ', 0.18955181155502235)
(' i= ', 3, ' best= ', 0.18947885536159983)
(' i= ', 4, ' best= ', 0.18824439650833161)
(' i= ', 5, ' best= ', 0.18824237700044519)
(' i= ', 6, ' best= ', 0.18650717739732442)
(' i= ', 7, ' best= ', 0.18567408772232108)
(' i= ', 8, ' best= ', 0.18551531264321136)
(' i= ', 9, ' best= ', 0.18504144178138185)
(' i= ', 10, ' best= ', 0.18248897653536789)
(' i= ', 11, ' best= ', 0.18108529416259517)


In [56]:
def data_to_TrainAndVal_knn(data,datao):
      
    data_starting_time=data["starting_time"].str.split(' ',expand=True)
    data_starting_time_date=data_starting_time[0].str.split('-',expand=True)
    data_starting_time_time=data_starting_time[1].str.split(':',expand=True)
    data["month"]=data_starting_time_date[1].apply(int)
    data["date"]=data_starting_time_date[2].apply(int)
    data["hour"]=data_starting_time_time[0].apply(int)
    data["minute"]=data_starting_time_time[1].apply(int)
    data["minute"]=(data["minute"]/20).apply(int)*20
    data["number_intersection_id"]=data["intersection_id"].replace(["A","B","C"],[1,2,3])
    data_std=datao.groupby(["intersection_id","tollgate_id","hour"])["travel_time"].std().reset_index().rename(columns={"travel_time":"travel_time_std"})
    data_mean=datao.groupby(["intersection_id","tollgate_id","hour"])["travel_time"].mean().reset_index().rename(columns={"travel_time":"travel_time_mean"})
    data=pd.merge(data,data_std,on=["intersection_id","tollgate_id","hour"])
    data=pd.merge(data,data_mean,on=["intersection_id","tollgate_id","hour"])
    
    data["holiday_before1"]=1*(((data["month"]==9)&(data["date"]==14))\
                            |((data["month"]==9)&(data["date"]==30)))
    data["holiday_before2"]=1*(((data["month"]==9)&(data["date"]==15))\
                            |((data["month"]==10)&(data["date"]==1)))
    data["holiday_after1"]=1*(((data["month"]==9)&(data["date"]==17))\
                            |((data["month"]==10)&(data["date"]==7)))
    data["holiday_after2"]=1*(((data["month"]==9)&(data["date"]==18))\
                            |((data["month"]==10)&(data["date"]==8)))
    data["holiday1"]=1*((data["month"]==9)&(data["date"]>15)&(data["date"]<19))
    data["holiday2"]=1*((data["month"]==10)&(data["date"]>0)&(data["date"]<8))
    data["workholiday"]=1*(((data["month"]==9)&(data["date"]==19))\
                        |((data["month"]==10)&(data["date"]==9))\
                        |((data["month"]==10)&(data["date"]==10)))
    data["holiday"]=data["holiday1"]+data["holiday2"]
   
    
    data["datetime"]=pd.to_datetime(data["starting_time"])
    data["weekday"]=data["datetime"].dt.weekday
    
    #one_hot A,B,C
    data["ifA"]=data["intersection_id"].replace(["A","B","C"],[1,0,0])
    data["ifB"]=data["intersection_id"].replace(["A","B","C"],[0,1,0])
    data["ifC"]=data["intersection_id"].replace(["A","B","C"],[0,0,1])

    #one_hot 1,2,3
    data["if1"]=data["tollgate_id"].replace([1,2,3],[1,0,0])
    data["if2"]=data["tollgate_id"].replace([1,2,3],[0,1,0])
    data["if3"]=data["tollgate_id"].replace([1,2,3],[0,0,1])
    
    #one_hot weekday
    data["wk1"]=data["weekday"].replace(range(0,7),[1,0,0,0,0,0,0])
    data["wk2"]=data["weekday"].replace(range(0,7),[0,1,0,0,0,0,0])
    data["wk3"]=data["weekday"].replace(range(0,7),[0,0,1,0,0,0,0])
    data["wk4"]=data["weekday"].replace(range(0,7),[0,0,0,1,0,0,0])
    data["wk5"]=data["weekday"].replace(range(0,7),[0,0,0,0,1,0,0])
    data["wk6"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,1,0])
    data["wk7"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,0,1])
    
    data["time_number"]=data["hour"]*60+data["minute"]
    data["date_number"]=(data["datetime"]-datetime(2016,7,19)).dt.days
    
#     data=data.iloc[:,[0,1,2]+range(3,len(data.T))]


    dtrain,dtest,dpred,ddata=select_val_knn(data)
    data_validation=dpred
    q=data_validation.groupby(["month","date","hour","minute","intersection_id","tollgate_id"])["travel_time"].mean()
    p=data_validation.drop(["travel_time","datetime","starting_time","time_number"],axis=1).drop_duplicates().sort(columns=["month","date","hour","minute","intersection_id","tollgate_id"])
    p["travel_time"]=q.values
    cols=list(p)
    cols.insert(2, cols.pop(cols.index('travel_time')))
    p= p.ix[:, cols]
    data_validation=p

    

    return dtrain,dtest,data_validation,ddata

In [57]:
data1 = pd.read_csv('../RawData/trajectories(table 5)_training.csv')
data2 = pd.read_csv('../RawData/trajectories(table_5)_training2.csv')
data3 = pd.read_csv('../RawData/trajectories(table 5)_test2.csv')
data=pd.concat([data1,data2,data3])
data=data.iloc[:,[0,1,3,5]]


dtrain,dtest,val,ddata=data_to_TrainAndVal_knn(data,data)
datao=ddata.copy()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:58: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [58]:
train=pd.concat([dtrain,dtest])
origin=train.copy()
t=data_avg(origin.copy())
train=t
train["datetime"]=pd.to_datetime(str(2016)+"-"+train["month"].astype(str)+'-'+train["date"].astype(str)\
            +' '+train["hour"].astype(str)+":"+train["minute"].astype(str))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [59]:
tmptw=twframe[twframe["datetime"]<pd.to_datetime(np.array([datetime(2016,10,31,19)]*len(twframe)).astype(str))]

In [60]:
complete_train=pd.merge(train,tmptw,how="right",on=["intersection_id","tollgate_id","datetime"])

In [61]:
nan_train=complete_train[complete_train["travel_time"].isnull()]
Nnan_train=complete_train[~(complete_train["travel_time"].isnull())]

In [62]:
nan_train

,intersection_id,tollgate_id,travel_time,month,date,hour,minute,number_intersection_id,travel_time_std,travel_time_mean,...,wk1,wk2,wk3,wk4,wk5,wk6,wk7,time_number,date_number,datetime
27757,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-19 00:00:00
27758,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-19 00:40:00
27759,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-19 01:00:00
27760,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-19 02:00:00
27761,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-19 03:00:00
27762,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-20 00:00:00
27763,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-20 09:20:00
27764,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-21 00:20:00
27765,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-21 00:40:00
27766,A,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-07-21 04:00:00


In [63]:
def data_to_TrainAndVal_nan(data,datao):
    

    data=data.copy().drop(["travel_time_std","travel_time_mean"],axis=1)
    data_starting_time=data["datetime"].astype(str).str.split(' ',expand=True)
    data_starting_time_date=data_starting_time[0].str.split('-',expand=True)
    data_starting_time_time=data_starting_time[1].str.split(':',expand=True)
    data["month"]=data_starting_time_date[1].apply(int)
    data["date"]=data_starting_time_date[2].apply(int)
    data["hour"]=data_starting_time_time[0].apply(int)
    data["minute"]=data_starting_time_time[1].apply(int)
    data["minute"]=(data["minute"]/20).apply(int)*20
    data["number_intersection_id"]=data["intersection_id"].replace(["A","B","C"],[1,2,3])
    data_std=datao.groupby(["intersection_id","tollgate_id","hour"])["travel_time"].std().reset_index().rename(columns={"travel_time":"travel_time_std"})
    data_mean=datao.groupby(["intersection_id","tollgate_id","hour"])["travel_time"].mean().reset_index().rename(columns={"travel_time":"travel_time_mean"})
    data=pd.merge(data,data_std,on=["intersection_id","tollgate_id","hour"])
    data=pd.merge(data,data_mean,on=["intersection_id","tollgate_id","hour"])
    
    data["holiday_before1"]=1*(((data["month"]==9)&(data["date"]==14))\
                            |((data["month"]==9)&(data["date"]==30)))
    data["holiday_before2"]=1*(((data["month"]==9)&(data["date"]==15))\
                            |((data["month"]==10)&(data["date"]==1)))
    data["holiday_after1"]=1*(((data["month"]==9)&(data["date"]==17))\
                            |((data["month"]==10)&(data["date"]==7)))
    data["holiday_after2"]=1*(((data["month"]==9)&(data["date"]==18))\
                            |((data["month"]==10)&(data["date"]==8)))
    data["holiday1"]=1*((data["month"]==9)&(data["date"]>15)&(data["date"]<19))
    data["holiday2"]=1*((data["month"]==10)&(data["date"]>0)&(data["date"]<8))
    data["workholiday"]=1*(((data["month"]==9)&(data["date"]==19))\
                        |((data["month"]==10)&(data["date"]==9))\
                        |((data["month"]==10)&(data["date"]==10)))
    data["holiday"]=data["holiday1"]+data["holiday2"]
    
    
    data["weekday"]=data["datetime"].dt.weekday
    
    #one_hot A,B,C
    data["ifA"]=data["intersection_id"].replace(["A","B","C"],[1,0,0])
    data["ifB"]=data["intersection_id"].replace(["A","B","C"],[0,1,0])
    data["ifC"]=data["intersection_id"].replace(["A","B","C"],[0,0,1])

    #one_hot 1,2,3
    data["if1"]=data["tollgate_id"].replace([1,2,3],[1,0,0])
    data["if2"]=data["tollgate_id"].replace([1,2,3],[0,1,0])
    data["if3"]=data["tollgate_id"].replace([1,2,3],[0,0,1])
    
    #one_hot weekday
    data["wk1"]=data["weekday"].replace(range(0,7),[1,0,0,0,0,0,0])
    data["wk2"]=data["weekday"].replace(range(0,7),[0,1,0,0,0,0,0])
    data["wk3"]=data["weekday"].replace(range(0,7),[0,0,1,0,0,0,0])
    data["wk4"]=data["weekday"].replace(range(0,7),[0,0,0,1,0,0,0])
    data["wk5"]=data["weekday"].replace(range(0,7),[0,0,0,0,1,0,0])
    data["wk6"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,1,0])
    data["wk7"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,0,1])
    
    data["time_number"]=data["hour"]*60+data["minute"]
    data["date_number"]=(data["datetime"]-datetime(2016,7,19)).dt.days
    data=data.iloc[:,[0,1,2]+range(3,len(data.T))]



    

    return data

In [64]:
nanfill_train=data_to_TrainAndVal_nan(nan_train,datao)

In [65]:
nanfill_train["travel_time"]=xlf.predict(nanfill_train.ix[:,list(z)].drop(['travel_time'],axis=1),
                                         ntree_limit=limit)

In [66]:
nanfill_train=adjust(nanfill_train,best_para)

In [67]:
nanfill_train

,intersection_id,tollgate_id,travel_time,month,date,hour,minute,number_intersection_id,holiday_before1,holiday_before2,...,wk3,wk4,wk5,wk6,wk7,time_number,date_number,datetime,travel_time_std,travel_time_mean
0,A,2,54.482658,7,19,0,0,1,0,0,...,0,0,0,0,0,0,0,2016-07-19 00:00:00,29.065737,56.773541
1,A,2,54.482658,7,19,0,40,1,0,0,...,0,0,0,0,0,40,0,2016-07-19 00:40:00,29.065737,56.773541
2,A,2,55.647675,7,20,0,0,1,0,0,...,1,0,0,0,0,0,1,2016-07-20 00:00:00,29.065737,56.773541
3,A,2,55.647675,7,21,0,20,1,0,0,...,0,1,0,0,0,20,2,2016-07-21 00:20:00,29.065737,56.773541
4,A,2,55.647675,7,21,0,40,1,0,0,...,0,1,0,0,0,40,2,2016-07-21 00:40:00,29.065737,56.773541
5,A,2,78.129776,7,23,0,0,1,0,0,...,0,0,0,1,0,0,4,2016-07-23 00:00:00,29.065737,56.773541
6,A,2,78.129776,7,23,0,20,1,0,0,...,0,0,0,1,0,20,4,2016-07-23 00:20:00,29.065737,56.773541
7,A,2,78.129776,7,23,0,40,1,0,0,...,0,0,0,1,0,40,4,2016-07-23 00:40:00,29.065737,56.773541
8,A,2,56.004410,7,25,0,40,1,0,0,...,0,0,0,0,0,40,6,2016-07-25 00:40:00,29.065737,56.773541
9,A,2,56.004410,7,26,0,40,1,0,0,...,0,0,0,0,0,40,7,2016-07-26 00:40:00,29.065737,56.773541


In [68]:
full_train=pd.concat([nanfill_train,Nnan_train]).ix[:,list(val)]
full_train.index=range(0,len(full_train))

In [69]:
full_train["datetime"]=pd.to_datetime(str(2016)+"-"+full_train["month"].astype(int).astype(str)+'-'+full_train["date"].astype(int).astype(str)\
            +' '+full_train["hour"].astype(int).astype(str)+":"+full_train["minute"].astype(int).astype(str))

In [70]:
full_train

,intersection_id,tollgate_id,travel_time,month,date,hour,minute,number_intersection_id,travel_time_std,travel_time_mean,...,if3,wk1,wk2,wk3,wk4,wk5,wk6,wk7,date_number,datetime
0,A,2,54.482658,7,19,0,0,1,29.065737,56.773541,...,0,0,1,0,0,0,0,0,0,2016-07-19 00:00:00
1,A,2,54.482658,7,19,0,40,1,29.065737,56.773541,...,0,0,1,0,0,0,0,0,0,2016-07-19 00:40:00
2,A,2,55.647675,7,20,0,0,1,29.065737,56.773541,...,0,0,0,1,0,0,0,0,1,2016-07-20 00:00:00
3,A,2,55.647675,7,21,0,20,1,29.065737,56.773541,...,0,0,0,0,1,0,0,0,2,2016-07-21 00:20:00
4,A,2,55.647675,7,21,0,40,1,29.065737,56.773541,...,0,0,0,0,1,0,0,0,2,2016-07-21 00:40:00
5,A,2,78.129776,7,23,0,0,1,29.065737,56.773541,...,0,0,0,0,0,0,1,0,4,2016-07-23 00:00:00
6,A,2,78.129776,7,23,0,20,1,29.065737,56.773541,...,0,0,0,0,0,0,1,0,4,2016-07-23 00:20:00
7,A,2,78.129776,7,23,0,40,1,29.065737,56.773541,...,0,0,0,0,0,0,1,0,4,2016-07-23 00:40:00
8,A,2,56.004410,7,25,0,40,1,29.065737,56.773541,...,0,1,0,0,0,0,0,0,6,2016-07-25 00:40:00
9,A,2,56.004410,7,26,0,40,1,29.065737,56.773541,...,0,0,1,0,0,0,0,0,7,2016-07-26 00:40:00


In [71]:
full_train=full_train.sort(columns=["intersection_id","tollgate_id","month","date","hour","minute"])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [72]:
full_train

,intersection_id,tollgate_id,travel_time,month,date,hour,minute,number_intersection_id,travel_time_std,travel_time_mean,...,if3,wk1,wk2,wk3,wk4,wk5,wk6,wk7,date_number,datetime
0,A,2,54.482658,7,19,0,0,1,29.065737,56.773541,...,0,0,1,0,0,0,0,0,0,2016-07-19 00:00:00
17514,A,2,58.050000,7,19,0,20,1,29.065737,56.773541,...,0,0,1,0,0,0,0,0,0,2016-07-19 00:20:00
1,A,2,54.482658,7,19,0,40,1,29.065737,56.773541,...,0,0,1,0,0,0,0,0,0,2016-07-19 00:40:00
102,A,2,54.482658,7,19,1,0,1,42.420071,58.726301,...,0,0,1,0,0,0,0,0,0,2016-07-19 01:00:00
17519,A,2,56.870000,7,19,1,20,1,42.420071,58.726301,...,0,0,1,0,0,0,0,0,0,2016-07-19 01:20:00
17523,A,2,77.740000,7,19,1,40,1,42.420071,58.726301,...,0,0,1,0,0,0,0,0,0,2016-07-19 01:40:00
232,A,2,54.482658,7,19,2,0,1,40.416861,59.179945,...,0,0,1,0,0,0,0,0,0,2016-07-19 02:00:00
17527,A,2,42.640000,7,19,2,20,1,40.416861,59.179945,...,0,0,1,0,0,0,0,0,0,2016-07-19 02:20:00
17529,A,2,40.173333,7,19,2,40,1,40.416861,59.179945,...,0,0,1,0,0,0,0,0,0,2016-07-19 02:40:00
360,A,2,53.330826,7,19,3,0,1,45.781885,58.020475,...,0,0,1,0,0,0,0,0,0,2016-07-19 03:00:00


In [73]:
train_knn=full_train[(full_train["hour"]==6)|(full_train["hour"]==7)|(full_train["hour"]==15)|(full_train["hour"]==16)].copy()
train_knn

,intersection_id,tollgate_id,travel_time,month,date,hour,minute,number_intersection_id,travel_time_std,travel_time_mean,...,if3,wk1,wk2,wk3,wk4,wk5,wk6,wk7,date_number,datetime
17551,A,2,46.356000,7,19,6,0,1,49.497381,59.054283,...,0,0,1,0,0,0,0,0,0,2016-07-19 06:00:00
17556,A,2,48.588000,7,19,6,20,1,49.497381,59.054283,...,0,0,1,0,0,0,0,0,0,2016-07-19 06:20:00
17560,A,2,66.642500,7,19,6,40,1,49.497381,59.054283,...,0,0,1,0,0,0,0,0,0,2016-07-19 06:40:00
17565,A,2,64.681000,7,19,7,0,1,39.187071,68.767064,...,0,0,1,0,0,0,0,0,0,2016-07-19 07:00:00
17570,A,2,85.676000,7,19,7,20,1,39.187071,68.767064,...,0,0,1,0,0,0,0,0,0,2016-07-19 07:20:00
17574,A,2,58.968889,7,19,7,40,1,39.187071,68.767064,...,0,0,1,0,0,0,0,0,0,2016-07-19 07:40:00
17683,A,2,79.420833,7,19,15,0,1,41.075873,71.065937,...,0,0,1,0,0,0,0,0,0,2016-07-19 15:00:00
17688,A,2,72.314000,7,19,15,20,1,41.075873,71.065937,...,0,0,1,0,0,0,0,0,0,2016-07-19 15:20:00
17694,A,2,58.741000,7,19,15,40,1,41.075873,71.065937,...,0,0,1,0,0,0,0,0,0,2016-07-19 15:40:00
17698,A,2,59.637500,7,19,16,0,1,39.887124,72.503117,...,0,0,1,0,0,0,0,0,0,2016-07-19 16:00:00


In [74]:
val_knn=data_avg(dtest)
val_knn

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,intersection_id,tollgate_id,travel_time,month,date,hour,minute,number_intersection_id,travel_time_std,travel_time_mean,...,if3,wk1,wk2,wk3,wk4,wk5,wk6,wk7,time_number,date_number
8538,A,2,29.742500,10,25,6,0,1,49.497381,59.054283,...,0,0,1,0,0,0,0,0,360,98
6562,A,3,84.930000,10,25,6,0,1,56.155640,112.759894,...,1,0,1,0,0,0,0,0,360,98
9029,B,1,103.810000,10,25,6,0,2,95.634350,130.427791,...,0,0,1,0,0,0,0,0,360,98
9473,B,3,273.550000,10,25,6,0,2,59.711535,102.714875,...,1,0,1,0,0,0,0,0,360,98
6948,C,1,231.340000,10,25,6,0,3,55.097820,158.501563,...,0,0,1,0,0,0,0,0,360,98
8542,A,2,56.415714,10,25,6,20,1,49.497381,59.054283,...,0,0,1,0,0,0,0,0,380,98
6564,A,3,108.198333,10,25,6,20,1,56.155640,112.759894,...,1,0,1,0,0,0,0,0,380,98
9030,B,1,88.710000,10,25,6,20,2,95.634350,130.427791,...,0,0,1,0,0,0,0,0,380,98
9474,B,3,106.990000,10,25,6,20,2,59.711535,102.714875,...,1,0,1,0,0,0,0,0,380,98
6949,C,1,136.270000,10,25,6,20,3,55.097820,158.501563,...,0,0,1,0,0,0,0,0,380,98


In [75]:
listmonth=[]
listdate=[]

In [76]:
tmpval=val_knn[val_knn["date"]==25].copy()
tmpval.index=range(0,len(tmpval))
start_time=datetime(2016,7,19)
bestmonth=0
bestday=0
bestscore=999
while(start_time<datetime(2016,10,25)):
    tmpscore=0
    scorelen=0
    for j in range(0,len(tmpval)):
        tmp=tmpval.iloc[j,:]
        tmptrain=train_knn[(train_knn["month"]==start_time.month)&\
                           (train_knn["date"]==start_time.day)&\
                           (train_knn["intersection_id"]==tmp["intersection_id"])&\
                           (train_knn["tollgate_id"]==tmp["tollgate_id"])&\
                           (train_knn["hour"]==tmp["hour"])&\
                           (train_knn["minute"]==tmp["minute"])]
        if(len(tmptrain)):
            tmpscore=tmpscore*scorelen/(scorelen+1)+abs(tmptrain["travel_time"].values-tmp["travel_time"])/(scorelen+1)
            scorelen=scorelen+1
            
    if(bestscore>tmpscore):
        bestscore=tmpscore
        bestday=tmptrain["date"]
        bestmonth=tmptrain["month"]
    start_time=start_time+timedelta(days=1)
print(bestmonth.values,"-",bestday.values)
listmonth=listmonth+[int(bestmonth.values[0])]
listdate=listdate+[int(bestday.values[0])]

(array([ 10.]), '-', array([ 13.]))


In [77]:
tmpval=val_knn[val_knn["date"]==26].copy()
tmpval.index=range(0,len(tmpval))
start_time=datetime(2016,7,19)
bestmonth=0
bestday=0
bestscore=999
while(start_time<datetime(2016,10,25)):
    tmpscore=0
    scorelen=0
    for j in range(0,len(tmpval)):
        tmp=tmpval.iloc[j,:]
        tmptrain=train_knn[(train_knn["month"]==start_time.month)&\
                           (train_knn["date"]==start_time.day)&\
                           (train_knn["intersection_id"]==tmp["intersection_id"])&\
                           (train_knn["tollgate_id"]==tmp["tollgate_id"])&\
                           (train_knn["hour"]==tmp["hour"])&\
                           (train_knn["minute"]==tmp["minute"])]
        if(len(tmptrain)):
            tmpscore=tmpscore*scorelen/(scorelen+1)+abs(tmptrain["travel_time"].values-tmp["travel_time"])/(scorelen+1)
            scorelen=scorelen+1
            
    if(bestscore>tmpscore):
        bestscore=tmpscore
        bestday=tmptrain["date"]
        bestmonth=tmptrain["month"]
    start_time=start_time+timedelta(days=1)
print(bestmonth.values,"-",bestday.values)
listmonth=listmonth+[int(bestmonth.values[0])]
listdate=listdate+[int(bestday.values[0])]

(array([ 10.]), '-', array([ 13.]))


In [78]:
tmpval=val_knn[val_knn["date"]==27].copy()
tmpval.index=range(0,len(tmpval))
start_time=datetime(2016,7,19)
bestmonth=0
bestday=0
bestscore=999
while(start_time<datetime(2016,10,25)):
    tmpscore=0
    scorelen=0
    for j in range(0,len(tmpval)):
        tmp=tmpval.iloc[j,:]
        tmptrain=train_knn[(train_knn["month"]==start_time.month)&\
                           (train_knn["date"]==start_time.day)&\
                           (train_knn["intersection_id"]==tmp["intersection_id"])&\
                           (train_knn["tollgate_id"]==tmp["tollgate_id"])&\
                           (train_knn["hour"]==tmp["hour"])&\
                           (train_knn["minute"]==tmp["minute"])]
        if(len(tmptrain)):
            tmpscore=tmpscore*scorelen/(scorelen+1)+abs(tmptrain["travel_time"].values-tmp["travel_time"])/(scorelen+1)
            scorelen=scorelen+1
            
    if(bestscore>tmpscore):
        bestscore=tmpscore
        bestday=tmptrain["date"]
        bestmonth=tmptrain["month"]
    start_time=start_time+timedelta(days=1)
print(bestmonth.values,"-",bestday.values)
listmonth=listmonth+[int(bestmonth.values[0])]
listdate=listdate+[int(bestday.values[0])]

(array([ 9.]), '-', array([ 29.]))


In [79]:
tmpval=val_knn[val_knn["date"]==28].copy()
tmpval.index=range(0,len(tmpval))
start_time=datetime(2016,7,19)
bestmonth=0
bestday=0
bestscore=999
while(start_time<datetime(2016,10,25)):
    tmpscore=0
    scorelen=0
    for j in range(0,len(tmpval)):
        tmp=tmpval.iloc[j,:]
        tmptrain=train_knn[(train_knn["month"]==start_time.month)&\
                           (train_knn["date"]==start_time.day)&\
                           (train_knn["intersection_id"]==tmp["intersection_id"])&\
                           (train_knn["tollgate_id"]==tmp["tollgate_id"])&\
                           (train_knn["hour"]==tmp["hour"])&\
                           (train_knn["minute"]==tmp["minute"])]
        if(len(tmptrain)):
            tmpscore=tmpscore*scorelen/(scorelen+1)+abs(tmptrain["travel_time"].values-tmp["travel_time"])/(scorelen+1)
            scorelen=scorelen+1
            
    if(bestscore>tmpscore):
        bestscore=tmpscore
        bestday=tmptrain["date"]
        bestmonth=tmptrain["month"]
    start_time=start_time+timedelta(days=1)
print(bestmonth.values,"-",bestday.values)
listmonth=listmonth+[int(bestmonth.values[0])]
listdate=listdate+[int(bestday.values[0])]

(array([ 9.]), '-', array([ 26.]))


In [80]:
tmpval=val_knn[val_knn["date"]==29].copy()
tmpval.index=range(0,len(tmpval))
start_time=datetime(2016,7,19)
bestmonth=0
bestday=0
bestscore=999
while(start_time<datetime(2016,10,25)):
    tmpscore=0
    scorelen=0
    for j in range(0,len(tmpval)):
        tmp=tmpval.iloc[j,:]
        tmptrain=train_knn[(train_knn["month"]==start_time.month)&\
                           (train_knn["date"]==start_time.day)&\
                           (train_knn["intersection_id"]==tmp["intersection_id"])&\
                           (train_knn["tollgate_id"]==tmp["tollgate_id"])&\
                           (train_knn["hour"]==tmp["hour"])&\
                           (train_knn["minute"]==tmp["minute"])]
        if(len(tmptrain)):
            tmpscore=tmpscore*scorelen/(scorelen+1)+abs(tmptrain["travel_time"].values-tmp["travel_time"])/(scorelen+1)
            scorelen=scorelen+1
            
    if(bestscore>tmpscore):
        bestscore=tmpscore
        bestday=tmptrain["date"]
        bestmonth=tmptrain["month"]
    start_time=start_time+timedelta(days=1)
print(bestmonth.values,"-",bestday.values)
listmonth=listmonth+[int(bestmonth.values[0])]
listdate=listdate+[int(bestday.values[0])]

(array([ 10.]), '-', array([ 2.]))


In [81]:
tmpval=val_knn[val_knn["date"]==30].copy()
tmpval.index=range(0,len(tmpval))
start_time=datetime(2016,7,19)
bestmonth=0
bestday=0
bestscore=999
while(start_time<datetime(2016,10,25)):
    tmpscore=0
    scorelen=0
    for j in range(0,len(tmpval)):
        tmp=tmpval.iloc[j,:]
        tmptrain=train_knn[(train_knn["month"]==start_time.month)&\
                           (train_knn["date"]==start_time.day)&\
                           (train_knn["intersection_id"]==tmp["intersection_id"])&\
                           (train_knn["tollgate_id"]==tmp["tollgate_id"])&\
                           (train_knn["hour"]==tmp["hour"])&\
                           (train_knn["minute"]==tmp["minute"])]
        if(len(tmptrain)):
            tmpscore=tmpscore*scorelen/(scorelen+1)+abs(tmptrain["travel_time"].values-tmp["travel_time"])/(scorelen+1)
            scorelen=scorelen+1
            
    if(bestscore>tmpscore):
        bestscore=tmpscore
        bestday=tmptrain["date"]
        bestmonth=tmptrain["month"]
    start_time=start_time+timedelta(days=1)
print(bestmonth.values,"-",bestday.values)
listmonth=listmonth+[int(bestmonth.values[0])]
listdate=listdate+[int(bestday.values[0])]

(array([ 9.]), '-', array([ 25.]))


In [82]:
tmpval=val_knn[val_knn["date"]==31].copy()
tmpval.index=range(0,len(tmpval))
start_time=datetime(2016,7,19)
bestmonth=0
bestday=0
bestscore=999
while(start_time<datetime(2016,10,25)):
    tmpscore=0
    scorelen=0
    for j in range(0,len(tmpval)):
        tmp=tmpval.iloc[j,:]
        tmptrain=train_knn[(train_knn["month"]==start_time.month)&\
                           (train_knn["date"]==start_time.day)&\
                           (train_knn["intersection_id"]==tmp["intersection_id"])&\
                           (train_knn["tollgate_id"]==tmp["tollgate_id"])&\
                           (train_knn["hour"]==tmp["hour"])&\
                           (train_knn["minute"]==tmp["minute"])]
        if(len(tmptrain)):
            tmpscore=tmpscore*scorelen/(scorelen+1)+abs(tmptrain["travel_time"].values-tmp["travel_time"])/(scorelen+1)
            scorelen=scorelen+1
            
    if(bestscore>tmpscore):
        bestscore=tmpscore
        bestday=tmptrain["date"]
        bestmonth=tmptrain["month"]
    start_time=start_time+timedelta(days=1)
print(bestmonth.values,"-",bestday.values)
listmonth=listmonth+[int(bestmonth.values[0])]
listdate=listdate+[int(bestday.values[0])]

(array([ 10.]), '-', array([ 6.]))


In [83]:
pd.DataFrame({"date":listdate,"month":listmonth}).iloc[:,[1,0]].to_csv("../Other/ValidationDay.csv",index=False)